In [171]:
import numpy as np
import pandas as pd

#md_addr = pd.read_csv("/home/eckson888/projekcik/data/MD_raw_addr2.csv",encoding='cp1250', sep=";")
md_addr = pd.read_csv("/home/eckson888/projekcik/data/KP_raw.csv",encoding='cp1250', sep=",")
drop_columns=["city_id","currency_code","timezone","flow","request_timestamp_utc","begintrip_timestamp_utc","dropoff_timestamp_utc",
              "promotion_usd","credits_usd","driver_upfront_fare_usd","original_fare_usd","base_fare_usd","surge_fare_usd","minimum_fare_roundup_usd",
              "per_mile_fare_usd","per_minute_fare_usd","cancellation_fee_usd","rounding_down_amount_usd","service_fee_usd","toll_amount_usd",
              "booking_fee_usd","earnings_boost_usd","wait_time_fare_usd","long_distance_surcharge_usd","vehicle_trip_number"]

for column in drop_columns:
    md_addr.drop(column, axis='columns', inplace=True)
    
md_addr.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5668 entries, 0 to 5667
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_type_name              5668 non-null   object 
 1   global_product_name            5668 non-null   object 
 2   request_timestamp_local        5668 non-null   object 
 3   begintrip_timestamp_local      5668 non-null   object 
 4   dropoff_timestamp_local        5668 non-null   object 
 5   eta                            5668 non-null   int64  
 6   surge_multiplier               5668 non-null   float64
 7   is_surged                      5668 non-null   bool   
 8   has_destination                5668 non-null   bool   
 9   is_pool_matched                5668 non-null   bool   
 10  trip_distance_miles            5668 non-null   float64
 11  trip_duration_seconds          5668 non-null   object 
 12  status                         5668 non-null   o

In [172]:
md_cancelled_trips = pd.DataFrame(columns=md_addr.columns)
cancelled_rows = md_addr[md_addr['is_completed'].astype(str) == 'False']
md_cancelled_trips = cancelled_rows.copy()
md_addr = md_addr[md_addr['is_completed'] != False]
 
md_cancelled_trips.head()  



,product_type_name,global_product_name,request_timestamp_local,begintrip_timestamp_local,dropoff_timestamp_local,eta,surge_multiplier,is_surged,has_destination,is_pool_matched,...,guaranteed_surge_multiplier,ufp_type,cancellation_type,is_directed_dispatch_trip,wait_duration_minutes,driver_trip_number,concierge_source_type,is_scheduled_trip,is_airport_trip,license_plate
6,UberX,UberX,2022-07-22 20:27:05.000,\N,\N,380,1.0,False,True,False,...,\N,ufp_honored,\N,False,\N,\N,\N,False,False,LU649GP
7,UberX,UberX,2022-07-22 20:39:20.000,\N,\N,181,1.0,False,True,False,...,\N,ufp_honored,\N,False,\N,\N,\N,False,False,LU649GP
35,UberX,UberX,2022-07-24 03:27:16.000,\N,\N,30,1.0,False,True,False,...,\N,ufp_honored,\N,False,\N,\N,\N,False,False,LU649GP
72,UberX,UberX,2022-07-27 04:04:14.000,\N,\N,736,1.0,False,True,False,...,\N,ufp_honored,\N,False,\N,\N,\N,False,False,LU649GP
96,Uber Pet,UberX,2022-07-28 01:22:08.000,\N,\N,355,1.0,False,True,False,...,\N,ufp_honored,\N,False,\N,\N,\N,False,False,LU649GP


In [173]:
#time feature engineering

md_addr.rename(columns={'eta': 'driver_pickup_eta'}, inplace=True)
    
new_time_features=["request_unix_time","begintrip_unix_time","dropoff_unix_time",
                   "trip_day_of_year","trip_day_of_week","trip_month_of_year","trip_hour_of_day","actual_wait_time"]

for c in new_time_features:
    md_addr[c]=0

md_addr['request_unix_time'] = pd.to_datetime(md_addr['request_timestamp_local']).astype('int64') // 10**9
md_addr['begintrip_unix_time'] = pd.to_datetime(md_addr['begintrip_timestamp_local']).astype('int64') // 10**9
md_addr['dropoff_unix_time'] = pd.to_datetime(md_addr['dropoff_timestamp_local']).astype('int64') // 10**9
md_addr['trip_day_of_year'] = pd.to_datetime(md_addr['begintrip_timestamp_local']).dt.day_of_year
md_addr['trip_day_of_week'] = pd.to_datetime(md_addr['begintrip_timestamp_local']).dt.day_of_week
md_addr['trip_month_of_year'] = pd.to_datetime(md_addr['begintrip_timestamp_local']).dt.month
md_addr['trip_hour_of_day'] = pd.to_datetime(md_addr['begintrip_timestamp_local']).dt.hour    
md_addr['actual_wait_time'] = (pd.to_datetime(md_addr['begintrip_timestamp_local']) - pd.to_datetime(md_addr['request_timestamp_local'])).dt.total_seconds()

md_addr.head()

,product_type_name,global_product_name,request_timestamp_local,begintrip_timestamp_local,dropoff_timestamp_local,driver_pickup_eta,surge_multiplier,is_surged,has_destination,is_pool_matched,...,is_airport_trip,license_plate,request_unix_time,begintrip_unix_time,dropoff_unix_time,trip_day_of_year,trip_day_of_week,trip_month_of_year,trip_hour_of_day,actual_wait_time
0,UberX,UberX,2022-07-22 18:06:28.000,2022-07-22 18:10:13.000,2022-07-22 18:26:09.000,144,1.0,False,True,False,...,False,LU649GP,1658513188,1658513413,1658514369,203,4,7,18,225.0
1,UberX,UberX,2022-07-22 18:33:40.000,2022-07-22 18:39:00.000,2022-07-22 18:48:37.000,241,1.0,False,True,False,...,False,LU649GP,1658514820,1658515140,1658515717,203,4,7,18,320.0
2,UberX,UberX,2022-07-22 18:50:22.000,2022-07-22 18:53:03.000,2022-07-22 19:02:11.000,100,1.0,False,True,False,...,False,LU649GP,1658515822,1658515983,1658516531,203,4,7,18,161.0
3,UberX,UberX,2022-07-22 19:16:42.000,2022-07-22 19:21:21.000,2022-07-22 19:26:01.000,225,1.0,False,True,False,...,False,LU649GP,1658517402,1658517681,1658517961,203,4,7,19,279.0
4,UberX,UberX,2022-07-22 19:30:58.000,2022-07-22 19:37:03.000,2022-07-22 19:43:58.000,288,1.0,False,True,False,...,False,LU649GP,1658518258,1658518623,1658519038,203,4,7,19,365.0


In [174]:
# data cleaning/dimension reduction
md_addr.drop("status", axis='columns', inplace=True)
md_addr.drop("is_completed", axis='columns', inplace=True)
md_addr.drop("global_product_name", axis='columns', inplace=True)
md_addr.drop("has_destination", axis='columns', inplace=True)
md_addr.drop("is_pool_matched", axis='columns', inplace=True)
md_addr.drop("is_flat_rate", axis='columns', inplace=True)
md_addr.drop("rounding_down_amount_local", axis='columns', inplace=True)
md_addr.drop("booking_fee_local", axis='columns', inplace=True)
md_addr.drop("toll_amount_local", axis='columns', inplace=True)
md_addr.drop("earnings_boost_local", axis='columns', inplace=True)
md_addr.drop("driver_cancellation_reason", axis='columns', inplace=True)
md_addr.drop("guaranteed_surge_multiplier", axis='columns', inplace=True)
md_addr.drop("cancellation_type", axis='columns', inplace=True)
md_addr.drop("is_directed_dispatch_trip", axis='columns', inplace=True)

md_addr['promotion_local'] = md_addr['promotion_local'].replace({'\\N': 0})
md_addr['service_fee_local'] = md_addr['service_fee_local'].replace({'\\N': 0})
md_addr['wait_time_fare_local'] = md_addr['wait_time_fare_local'].replace({'\\N': 0})
md_addr['long_distance_surcharge_local'] = md_addr['long_distance_surcharge_local'].replace({'\\N': 0})
md_addr['wait_duration_minutes'] = md_addr['wait_duration_minutes'].replace({'\\N': 0})
md_addr['minimum_fare_roundup_local'] = md_addr['minimum_fare_roundup_local'].replace({'\\N': 0})
md_addr['concierge_source_type'] = md_addr['concierge_source_type'].replace({'\\N': 'none'})
md_addr['credits_local'] = md_addr['credits_local'].replace({'\\N': 0})

In [175]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(md_addr)


,product_type_name,request_timestamp_local,begintrip_timestamp_local,dropoff_timestamp_local,driver_pickup_eta,surge_multiplier,is_surged,trip_distance_miles,trip_duration_seconds,is_cash_trip,promotion_local,credits_local,has_driver_upfront_fare,driver_upfront_fare_local,original_fare_local,base_fare_local,surge_fare_local,minimum_fare_roundup_local,per_mile_fare_local,per_minute_fare_local,cancellation_fee_local,service_fee_local,fare_distance_miles,fare_duration_minutes,wait_time_fare_local,service_fee,is_multidestination,driver_surge_multiplier,long_distance_surcharge_local,ufp_type,wait_duration_minutes,driver_trip_number,concierge_source_type,is_scheduled_trip,is_airport_trip,license_plate,request_unix_time,begintrip_unix_time,dropoff_unix_time,trip_day_of_year,trip_day_of_week,trip_month_of_year,trip_hour_of_day,actual_wait_time
0,UberX,2022-07-22 18:06:28.000,2022-07-22 18:10:13.000,2022-07-22 18:26:09.000,144,1.0,False,6.169521,956,False,-2.28,0,False,21.08,20.49,4.4,0,0,2.6554176,0.17,0.0,0,6.17,15.933333333333334,0,0.19,False,1.0,0,ufp_honored,0,725,none,False,False,LU649GP,1658513188,1658513413,1658514369,203,4,7,18,225.0
1,UberX,2022-07-22 18:33:40.000,2022-07-22 18:39:00.000,2022-07-22 18:48:37.000,241,1.0,False,2.552311,538,True,-1.28,0,False,11.86,11.53,4.4,0,0,2.6554176,0.17,0.0,0,2.55,8.966666666666667,0,0.19,False,1.0,0,ufp_honored,0,726,none,False,False,LU649GP,1658514820,1658515140,1658515717,203,4,7,18,320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5665,Comfort,2025-04-06 02:57:01.000,2025-04-06 02:58:48.000,2025-04-06 03:06:08.000,115,1.0,False,3.938313,440,False,-16.61,0,True,20.76,20.76,7.09,0,0,2.91291264,0.27,0.0,-4.15,3.94,7.33333333,0,0.2,False,1.0,0,ufp_honored,0,6165,none,False,False,LU630TE,1743908221,1743908328,1743908768,96,6,4,2,107.0
5666,UberX,2025-04-07 08:12:40.000,2025-04-07 08:16:13.000,2025-04-07 08:23:30.000,195,1.0,False,1.832946,437,False,-2.6,0,True,13,13,5.45,0,0,2.23698816,0.21,0.0,-2.6,1.83,7.28333333,0,0.2,False,1.0,0,ufp_honored,0,6166,none,False,False,LU630TE,1744013560,1744013773,1744014210,97,0,4,8,213.0


In [176]:
for c in md_addr.columns:
    print(f"\ncolumn: {c}")
    print(md_addr[c].value_counts())


column: product_type_name
product_type_name
UberX             3675
Green             1598
Uber Pet            76
Comfort             59
UberX Priority      34
Name: count, dtype: int64

column: request_timestamp_local
request_timestamp_local
2025-04-07 08:12:40.000    1
2022-07-22 18:06:28.000    1
2025-04-03 14:55:18.000    1
2025-04-03 14:41:49.000    1
2025-04-03 14:17:24.000    1
                          ..
2022-07-22 21:15:01.000    1
2022-07-22 19:53:54.000    1
2022-07-22 19:30:58.000    1
2022-07-22 19:16:42.000    1
2022-07-22 18:50:22.000    1
Name: count, Length: 5442, dtype: int64

column: begintrip_timestamp_local
begintrip_timestamp_local
2025-04-07 08:16:13.000    1
2022-07-22 18:10:13.000    1
2025-04-03 15:01:36.000    1
2025-04-03 14:44:51.000    1
2025-04-03 14:20:19.000    1
                          ..
2022-07-22 21:21:59.000    1
2022-07-22 19:58:50.000    1
2022-07-22 19:37:03.000    1
2022-07-22 19:21:21.000    1
2022-07-22 18:53:03.000    1
Name: count, Lengt